## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/Maria/Desktop/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Coihaique,-45.5752,-72.0662,68.00,42,0,8.05
1,1,Kurchum,48.5722,83.6542,-20.34,83,0,2.93
2,2,Kaitangata,-46.2817,169.8464,66.00,73,92,4.00
3,3,Barrow,71.2906,-156.7887,-11.20,77,1,13.80
4,4,Punta Arenas,-53.1500,-70.9167,55.40,71,0,12.66


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [4]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Coihaique,-45.5752,-72.0662,68.00,42,0,8.05
2,2,Kaitangata,-46.2817,169.8464,66.00,73,92,4.00
9,9,Los Llanos De Aridane,28.6585,-17.9182,64.40,55,75,16.11
25,25,Ukiah,39.1502,-123.2078,69.01,45,1,4.68
30,30,Kidal,18.4411,1.4078,68.83,21,60,11.25
35,35,Raga,8.4596,25.6780,64.80,26,93,3.83
39,39,Lodja,-3.4833,23.4333,69.57,95,100,0.22
44,44,Mungwi,-10.1732,31.3694,62.56,96,100,4.05
47,47,Mar Del Plata,-38.0023,-57.5575,69.01,73,0,14.97
51,51,Hermanus,-34.4187,19.2345,64.00,85,0,11.99


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       97
City          97
Lat           97
Lng           97
Max Temp      97
Humidity      97
Cloudiness    97
Wind Speed    97
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Max Temp", "Cloudiness", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Cloudiness,Lat,Lng,Hotel Name
0,Coihaique,68.00,0,-45.5752,-72.0662,
2,Kaitangata,66.00,92,-46.2817,169.8464,
9,Los Llanos De Aridane,64.40,75,28.6585,-17.9182,
25,Ukiah,69.01,1,39.1502,-123.2078,
30,Kidal,68.83,60,18.4411,1.4078,
35,Raga,64.80,93,8.4596,25.6780,
39,Lodja,69.57,100,-3.4833,23.4333,
44,Mungwi,62.56,100,-10.1732,31.3694,
47,Mar Del Plata,69.01,0,-38.0023,-57.5575,
51,Hermanus,64.00,0,-34.4187,19.2345,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/Maria/Desktop/Weather_Database/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Cloudiness</dt><dd>{Cloudiness}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))